# Importing libraries

In [2]:
import numpy as np
import pickle
import os
import sys

## Defining constants

In [3]:
STATE_SIZE = (25,25)
ACTION_SIZE = 4

## Defining loading and saving of files

In [8]:
def load(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [59]:
def save(filename, Q_table):
    with open(filename, 'wb') as f:
        pickle.dump(Q_table, f)

## Defining the static environment



In [46]:
class DroneGridEnv():

    def __init__(self, grid):

        self.grid = grid
        self.grid_size = np.array(grid).shape
        self.observation_space = (self.grid_size[0]), (self.grid_size[1])
        self.action_space = [0, 1, 2, 3] # 4 discrete actions: 0 = up, 1 = down, 2 = left, 3 = right
        self.start_pos = (0, 0)  # Starting position at top left corner
        self.goal_pos = (self.grid_size[0] - 1, self.grid_size[1] - 1)  # Goal position at bottom right corner
        self.current_pos = self.start_pos  # Initialize current position

    def reset(self):
        self.current_pos = self.start_pos  # Reset current position to start position
        return self.current_pos  # Return initial state

    def step(self, action):

        assert action in self.action_space, f"Invalid action {action}"  # Check if action is valid

        # Define movement based on action
        if action == 0:  # Up
            new_pos = (self.current_pos[0], self.current_pos[1] - 1)
        elif action == 1:  # Down
            new_pos = (self.current_pos[0], self.current_pos[1] + 1)
        elif action == 2:  # Left
            new_pos = (self.current_pos[0] - 1, self.current_pos[1])
        elif action == 3:  # Right
            new_pos = (self.current_pos[0] + 1, self.current_pos[1])

        # Check if new position is within bounds and not an obstacle
        if 0 <= new_pos[0] < self.grid_size[0] and 0 <= new_pos[1] < self.grid_size[1] and self.grid[new_pos[1]][new_pos[0]] != 1:

            self.current_pos = new_pos  # Update current position

            # Check if goal state is reached
            done = (self.current_pos == self.goal_pos)

            # Calculate reward
            if done:
                reward = 100.0  # Positive reward for reaching the goal

            elif self.grid[new_pos[1]][new_pos[0]] == 1:
                reward = -100 # Negative reward for going in a wall

            else:
                reward = 0 #Negative reward for non-goal state

        else:
            done = False
            reward = -100  # Negative reward for going out of bounds

        return self.current_pos, reward, done

Loading a personalized map

In [41]:
map = load('map_simple.pkl')
print(map)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Creating an instance of the environment through the loaded map

In [48]:
environment = DroneGridEnv(map)
print(environment.observation_space)
print(environment.action_space)


(25, 25)
[0, 1, 2, 3]


---
# Q-Learning
---


In [62]:
def q_learning(env, alpha=1, gamma=0.9,  epsilon=1, epsilon_decay=0.005):

    Q = np.zeros((env.grid_size[0]*env.grid_size[1], len(env.action_space)), dtype=np.float32) #Initialize the Q table to all 0s

    for e in range(1000): #Run 1k training runs

        state = env.reset() #Part of OpenAI where you need to reset at the start of each run
        total_reward = 0 #Set initial reward to 0

        while True: #Loop until done == True
            #IF random number is less than epsilon grab the random action else grab the argument max of Q[state]

            current_state_index = env.current_pos[0] + env.current_pos[1]*env.observation_space[0]

            if np.random.uniform(0,1) < epsilon:
                action = np.random.choice(range(len(env.action_space)))

            else:
                action = np.argmax(Q[current_state_index])


            posp1, reward, done = env.step(action) #Send your action to OpenAI and get back the tuple

            state_tp1_index = posp1[0] + posp1[1]*env.observation_space[0]

            total_reward += reward #Increment your reward

            Q[current_state_index][action] = Q[current_state_index][action] + alpha * (reward + gamma * np.max(Q[state_tp1_index]) - Q[current_state_index][action])

             #Make sure to keep random at 10%

            if done:
                print(f"Episode: {e}, Reward: {total_reward}")
                break

        epsilon *= np.exp(-epsilon_decay)

    return Q

Effective running of the Q-Learning and saving of the trained Q-Table

In [64]:
q = q_learning(environment)
save('Q_Table - Q-Learning.pkl', q)

Episode: 0, Reward: -147800.0
Episode: 1, Reward: -36400.0
Episode: 2, Reward: -33900.0
Episode: 3, Reward: -16600.0
Episode: 4, Reward: -38500.0
Episode: 5, Reward: -100300.0
Episode: 6, Reward: -63400.0
Episode: 7, Reward: -58200.0
Episode: 8, Reward: -23900.0
Episode: 9, Reward: -43400.0
Episode: 10, Reward: -15200.0
Episode: 11, Reward: -99600.0
Episode: 12, Reward: -72400.0
Episode: 13, Reward: -10800.0
Episode: 14, Reward: -29500.0
Episode: 15, Reward: -49600.0
Episode: 16, Reward: -105800.0
Episode: 17, Reward: -16400.0
Episode: 18, Reward: -47900.0
Episode: 19, Reward: -11400.0
Episode: 20, Reward: -13700.0
Episode: 21, Reward: -9800.0
Episode: 22, Reward: -6900.0
Episode: 23, Reward: -23100.0
Episode: 24, Reward: -15200.0
Episode: 25, Reward: -14200.0
Episode: 26, Reward: -3900.0
Episode: 27, Reward: -4500.0
Episode: 28, Reward: -23300.0
Episode: 29, Reward: -2500.0
Episode: 30, Reward: -5000.0
Episode: 31, Reward: -8600.0
Episode: 32, Reward: -4100.0
Episode: 33, Reward: -240

Loading Q-Learning trained Q-Table and checking if successful

In [61]:
trained_q = load('Q_Table - Q-Learning.pkl')
print(trained_q)

[[-99.36373      0.70696515 -99.36373      0.70696515]
 [-99.29304      0.78551686   0.6362686    0.78551686]
 [-99.214485     0.8727965    0.70696515   0.8727965 ]
 ...
 [ 72.9        -19.          72.9         90.        ]
 [ 81.         -10.          81.         100.        ]
 [  0.           0.           0.           0.        ]]


---

# SARSA

---

In [65]:
def sarsa(env, alpha=1, gamma=0.9,  epsilon=1, epsilon_decay=0.005):

    def compute_action(current_state, Q_table):

        if np.random.uniform(0,1) < epsilon:
            return np.random.choice(range(len(env.action_space)))

        else:
            return np.argmax(Q_table[current_state]) # Choose the best action according to the Q_table in this state

    Q = np.zeros((env.grid_size[0]*env.grid_size[1], len(env.action_space)), dtype=np.float32) #Initialize the Q table to all 0s

    for e in range(1000): #Run 1k training runs

        state = env.reset() #Part of OpenAI where you need to reset at the start of each run
        total_reward = 0 #Set initial reward to 0

        while True: #Loop until done == True
            #IF random number is less than epsilon grab the random action else grab the argument max of Q[state]

            current_state_index = env.current_pos[0] + env.current_pos[1]*env.observation_space[0] # Obtain the index of the state

            action = compute_action(current_state_index, Q) # Compute the action for the current state using Q-Table

            posp1, reward, done = env.step(action) # Send the action to the environment and obtain the new position, the reward and the termination flag

            state_tp1_index = posp1[0] + posp1[1]*env.observation_space[0] # Compute the index of the state at t+1
            action_tp1 = compute_action(state_tp1_index, Q) # Compute the action for the next state using Q-Table

            total_reward += reward # Increment the reward

            Q[current_state_index][action] = Q[current_state_index][action] + alpha * (reward + gamma*Q[state_tp1_index][action_tp1] - Q[current_state_index][action])

             #Make sure to keep random at 10%

            if done:
                print(f"Episode: {e}, Reward: {total_reward}")
                break

        epsilon *= np.exp(-epsilon_decay)

    return Q

Effective running of SARSA and saving of the trained Q-Table

In [66]:
s = sarsa(environment)
save('Q_Table - SARSA.pkl', s)

Episode: 0, Reward: -37400.0
Episode: 1, Reward: -33600.0
Episode: 2, Reward: -49800.0
Episode: 3, Reward: -2600.0
Episode: 4, Reward: -15700.0
Episode: 5, Reward: -6900.0
Episode: 6, Reward: -70000.0
Episode: 7, Reward: -121900.0
Episode: 8, Reward: -92400.0
Episode: 9, Reward: -110100.0
Episode: 10, Reward: -26600.0
Episode: 11, Reward: -18600.0
Episode: 12, Reward: -4900.0
Episode: 13, Reward: -32500.0
Episode: 14, Reward: -52600.0
Episode: 15, Reward: -16300.0
Episode: 16, Reward: -44800.0
Episode: 17, Reward: -32300.0
Episode: 18, Reward: -23900.0
Episode: 19, Reward: -19400.0
Episode: 20, Reward: -30700.0
Episode: 21, Reward: -3500.0
Episode: 22, Reward: -23800.0
Episode: 23, Reward: -45500.0
Episode: 24, Reward: -15200.0
Episode: 25, Reward: -41400.0
Episode: 26, Reward: -80200.0
Episode: 27, Reward: -6200.0
Episode: 28, Reward: -21400.0
Episode: 29, Reward: -15200.0
Episode: 30, Reward: -22400.0
Episode: 31, Reward: -6800.0
Episode: 32, Reward: -5500.0
Episode: 33, Reward: -199

Loading SARSA trained Q-Table and checking if successful

In [67]:
trained_s = load('Q_Table - SARSA.pkl')
print(trained_s)

[[-100.           0.        -100.         -90.       ]
 [-100.         -90.87641      0.         -90.       ]
 [-100.        -100.97379      0.          -5.2334766]
 ...
 [  72.9        -65.13216     72.9         90.       ]
 [  81.         -10.          81.         100.       ]
 [   0.           0.           0.           0.       ]]


---

# Alternating Q-Learning / SARSA

---

---

# Deep Q-Learning

---

---

# Spatial Computing for Path Planning - SCPP - Personalized algorithm

---